In [1]:
# Install required packages
!pip install deep-translator
!pip install langdetect  
!pip install transformers
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!python -m spacy download ru_core_news_sm
!python -m spacy download uk_core_news_sm

# Import necessary libraries
import pandas as pd
import re
import unicodedata
import emoji
import spacy
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from langdetect import detect
from deep_translator import GoogleTranslator
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=2c0ce3e850f30a4163b3697e40d285fe733fcb2127d03c40bfeb58b5d0effe62
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 39.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 67.4 MB/s eta 0:00:00:00:010:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the pac

2025-04-14 07:16:27.746403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744614987.977856      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744614988.052453      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import pandas as pd
import numpy as np
import re
import unicodedata
import emoji
import spacy
import nltk
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

from nltk.corpus import stopwords
from langdetect import detect
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, hamming_loss, accuracy_score
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    get_linear_schedule_with_warmup
)
from datasets import Dataset
import logging

In [3]:
# Add these imports to your existing imports
import random
import nltk
from nltk.corpus import wordnet
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm

# Download necessary NLTK resources
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)
nltk.download('omw-1.4', quiet=True)

True

In [4]:
# Load datasets
train_df = pd.read_csv('/kaggle/input/finaldataset/NLP - exp(Encoded UTF-8).csv')
print("Training Dataset Shape:", train_df.shape)
print("Training Dataset Columns:", train_df.columns)
print("Training Dataset Head:\n", train_df.head())

# Load test dataset without skipping rows
test_df = pd.read_csv('/kaggle/input/finaldataset/final test.csv')
print("Test Dataset Shape (raw):", test_df.shape)
print("Test Dataset Columns (raw):", test_df.columns)
print("Test Dataset Head (raw):\n", test_df.head())

Training Dataset Shape: (3822, 6)
Training Dataset Columns: Index(['id', 'content', 'lang', 'manipulative', 'techniques', 'trigger_words'], dtype='object')
Training Dataset Head:
                                      id  \
0  0bb0c7fa-101b-4583-a5f9-9d503339141c   
1  7159f802-6f99-4e9d-97bd-6f565a4a0fae   
2  e6a427f1-211f-405f-bd8b-70798458d656   
3  1647a352-4cd3-40f6-bfa1-d87d42e34eea   
4  9c01de00-841f-4b50-9407-104e9ffb03bf   

                                             content lang  manipulative  \
0  Новий огляд мапи DeepState від російського вій...   uk          True   
1  Недавно 95 квартал жёстко поглумился над русск...   ru          True   
2  🤩\nТим часом йде евакуація Бєлгородського авто...   uk          True   
3  В Україні найближчим часом мають намір посилит...   uk         False   
4  Расчёты 122-мм САУ 2С1 "Гвоздика" 132-й бригад...   ru          True   

                             techniques  \
0        ['euphoria' 'loaded_language']   
1  ['loaded_language' 'c

In [5]:
# Check and fix column names if 'content' is missing
if 'content' not in train_df.columns:
    print("Error: 'content' not found in train_df. Please check the CSV file structure.")
    raise KeyError("'content' column missing in train_df")

if 'content' not in test_df.columns:
    print("Adjusting test_df columns because 'content' is missing...")
    # Assume the second column is 'content' if it exists
    if len(test_df.columns) >= 2:
        test_df.columns = ['id', 'content'] 
    else:
        print("Error: test_df has insufficient columns. Expected at least 'id' and 'content'.")
        raise KeyError("'content' column missing or test_df malformed")
    print("Adjusted Test Dataset Columns:", test_df.columns)
    print("Adjusted Test Dataset Head:\n", test_df.head())

# Add step to filter test_df to exactly 5735 rows by removing invalid rows
print("Initial Test Data Shape:", test_df.shape)
print("First few rows of test_df:\n", test_df.head())

# Remove rows where the 'id' column is invalid (e.g., contains the string "id" or is not a proper data row)
# This assumes the invalid row has 'id' as the value in the 'id' column, which is likely a duplicate header
test_df = test_df[test_df['id'] != 'id']

# If there are still more than 5735 rows, keep only the first 5735 valid rows
# This ensures we don't accidentally remove too many rows if there are other invalid entries
if len(test_df) > 5735:
    test_df = test_df.iloc[:5735]
elif len(test_df) < 5735:
    raise ValueError(f"Test data has fewer than 5735 rows after filtering ({len(test_df)} rows). Check test.csv file.")

print("Test Data Shape after filtering:", test_df.shape)

# Verify the row count
if len(test_df) != 5735:
    raise ValueError(f"Test data should have 5735 rows, but found {len(test_df)} rows after filtering. Check test.csv file.")
    
# Text cleaning function
def clean_text(text):
    text = str(text).lower()
    text = unicodedata.normalize("NFKD", text)
    text = "".join([c for c in text if not unicodedata.combining(c)])
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[^a-zа-яёіїєґ0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Apply cleaning
train_df["content"] = train_df["content"].apply(clean_text)
test_df["content"] = test_df["content"].apply(clean_text)
print("Cleaned Train Dataset Head:\n", train_df[["content"]].head())
print("Cleaned Test Dataset Head:\n", test_df[["content"]].head())

Adjusting test_df columns because 'content' is missing...
Adjusted Test Dataset Columns: Index(['id', 'content'], dtype='object')
Adjusted Test Dataset Head:
                                      id  \
0                                    id   
1  521cd2e8-dd9f-42c4-98ba-c0c8890ff1ba   
2  9b2a61e4-d14e-4ff7-b304-e73d720319bf   
3  f0f1c236-80a8-4d25-b30c-a420a39be632   
4  31ea05ba-2c2b-4b84-aba7-f3cf6841b204   

                                             content  
0                                            content  
1  Они просрали нашу технику, положили кучу людей...  
2  ❗️\nКитай предлагает отдать оккупированные тер...  
3  Сегодня будет ровно 6 месяцев с этого обещания...  
4  ⚡️\nІзраїль вперше у світі збив балістичну рак...  
Initial Test Data Shape: (5736, 2)
First few rows of test_df:
                                      id  \
0                                    id   
1  521cd2e8-dd9f-42c4-98ba-c0c8890ff1ba   
2  9b2a61e4-d14e-4ff7-b304-e73d720319bf   
3  f0f1c236-80a8-

In [6]:
# Define manipulation techniques
techniques = [
    "loaded_language", "glittering_generalities", "euphoria", "appeal_to_fear", "fud",
    "bandwagon", "thought_terminating_cliche", "whataboutism", "cherry_picking", "straw_man"
]
num_classes = len(techniques)

# Convert techniques column to multilabel binary format
def techniques_to_binary(tech_str):
    if pd.isna(tech_str):
        return [0] * num_classes
    tech_str = tech_str.strip("[]").replace("'", "").replace('"', '')
    tech_list = [tech.strip() for tech in tech_str.split() if tech.strip()]
    if not tech_list:
        tech_list = [tech.strip() for tech in tech_str.split(',') if tech.strip()]
    return [1 if tech in tech_list else 0 for tech in techniques]

train_df["labels"] = train_df["techniques"].apply(techniques_to_binary)

# Split dataset
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)


In [7]:
!pip install nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.4 MB/s eta 0:00:00ta 0:00:01


In [8]:
from nlpaug.augmenter.word import SynonymAug, BackTranslationAug, ContextualWordEmbsAug
import nlpaug.flow as naf
import nltk

# Download necessary NLTK resources
nltk.download('wordnet')

def augment_minority_classes_improved(df, techniques, min_samples=500):
    print("Starting improved data augmentation for minority classes...")
    original_rows = df.to_dict('records')
    augmented_rows = []
    labels_array = np.array([row['labels'] for row in original_rows])
    class_counts = labels_array.sum(axis=0)
    print("Original class counts:", {techniques[i]: count for i, count in enumerate(class_counts)})
    
    # Setup augmenters - customize per language if needed
    synonym_aug = SynonymAug(aug_src='wordnet')  # For English content
    
    # Optional: Add back-translation augmenter if you have internet access
    # back_trans_aug = BackTranslationAug(
    #    from_model_name='Helsinki-NLP/opus-mt-ru-en', 
    #    to_model_name='Helsinki-NLP/opus-mt-en-ru'
    # )
    
    # Contextual word embeddings augmenter (works better for maintaining propaganda techniques)
    contextual_aug = ContextualWordEmbsAug(
        model_path='xlm-roberta-base',
        action="substitute", 
        aug_p=0.1  # Probability of changing each word
    )
    
    # Create augmentation flow - combine multiple techniques
    aug_flow = naf.Sometimes([
        synonym_aug,  
        contextual_aug,
        # back_trans_aug  # Uncomment if available
    ], aug_p=0.7)
    
    for class_idx, count in enumerate(class_counts):
        if count == 0:
            print(f"Skipping {techniques[class_idx]} as it has 0 samples")
            continue
            
        if count < min_samples:
            class_rows = [row for row in original_rows if row['labels'][class_idx] == 1]
            needed = min_samples - count
            print(f"Augmenting {techniques[class_idx]}: Current={count}, Target={min_samples}, Generating {needed} new samples")
            
            for _ in range(needed):
                row = random.choice(class_rows)
                new_row = row.copy()
                
                # Apply language-specific augmentation based on detected language
                if row['lang'] in ['ru', 'uk']:
                    new_row['content'] = contextual_aug.augment(row['content'])
                else:
                    new_row['content'] = aug_flow.augment(row['content'])
                    
                augmented_rows.append(new_row)
    
    combined_data = original_rows + augmented_rows
    new_df = pd.DataFrame(combined_data)
    new_labels_array = np.array(new_df["labels"].tolist())
    new_class_counts = new_labels_array.sum(axis=0)
    print("New class counts after augmentation:", {techniques[i]: count for i, count in enumerate(new_class_counts)})
    return new_df
# Apply augmentation
balanced_train_df = augment_minority_classes_improved(train_df, techniques)

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Starting improved data augmentation for minority classes...
Original class counts: {'loaded_language': 1581, 'glittering_generalities': 386, 'euphoria': 385, 'appeal_to_fear': 242, 'fud': 310, 'bandwagon': 122, 'thought_terminating_cliche': 0, 'whataboutism': 124, 'cherry_picking': 415, 'straw_man': 113}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Augmenting glittering_generalities: Current=386, Target=500, Generating 114 new samples
Augmenting euphoria: Current=385, Target=500, Generating 115 new samples
Augmenting appeal_to_fear: Current=242, Target=500, Generating 258 new samples
Augmenting fud: Current=310, Target=500, Generating 190 new samples
Augmenting bandwagon: Current=122, Target=500, Generating 378 new samples
Skipping thought_terminating_cliche as it has 0 samples
Augmenting whataboutism: Current=124, Target=500, Generating 376 new samples
Augmenting cherry_picking: Current=415, Target=500, Generating 85 new samples
Augmenting straw_man: Current=113, Target=500, Generating 387 new samples
New class counts after augmentation: {'loaded_language': 3025, 'glittering_generalities': 718, 'euphoria': 609, 'appeal_to_fear': 737, 'fud': 887, 'bandwagon': 604, 'thought_terminating_cliche': 0, 'whataboutism': 598, 'cherry_picking': 1018, 'straw_man': 639}


In [9]:
import torch
import numpy as np

# Define device (add this before your function calls)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Calculate class weights using balanced dataset
def calculate_class_weights(df, techniques):
    labels_array = np.array(df["labels"].tolist())
    class_counts = labels_array.sum(axis=0)
    class_counts = np.array([max(1, count) for count in class_counts])
    weights = len(df) / (len(techniques) * class_counts)
    weights = weights / weights.sum() * len(techniques)
    print("Class weights:", {techniques[i]: round(weight, 2) for i, weight in enumerate(weights)})
    return torch.tensor(weights, dtype=torch.float).to(device)

class_weights = calculate_class_weights(balanced_train_df, techniques)

# Weighted BCE loss function
def weighted_bce_loss(outputs, targets):
    probs = torch.sigmoid(outputs)
    loss = 0
    for i in range(targets.size(1)):
        weight = class_weights[i]
        loss += weight * (-targets[:, i] * torch.log(probs[:, i] + 1e-7) -
                         (1 - targets[:, i]) * torch.log(1 - probs[:, i] + 1e-7))
    return loss.mean()


Class weights: {'loaded_language': 0.0, 'glittering_generalities': 0.01, 'euphoria': 0.02, 'appeal_to_fear': 0.01, 'fud': 0.01, 'bandwagon': 0.02, 'thought_terminating_cliche': 9.88, 'whataboutism': 0.02, 'cherry_picking': 0.01, 'straw_man': 0.02}


In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
# Dataset class
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=128):
        self.df = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.texts = self.df["content"].values
        self.labels = self.df["labels"].values if "labels" in self.df.columns else [[0] * num_classes] * len(self.df)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = np.array(self.labels[idx], dtype=np.float32)
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.float)
        }

# Model and tokenizer setup
model_name = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_classes,
    ignore_mismatched_sizes=True
)
model.to(device)

# Create DataLoaders
train_dataset = TextDataset(balanced_train_df, tokenizer)
val_dataset = TextDataset(val_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

# Verify setup
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training dataset size: 4960
Validation dataset size: 765


In [11]:
import torch
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import nlpaug.augmenter.word as naw
import nlpaug.flow as naf
import nltk
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score

# [Previous code up to DataLoader creation remains unchanged...]

# Focal Loss to better handle class imbalance
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean', class_weights=None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.class_weights = class_weights  # Optional: Add class weights

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        
        if self.class_weights is not None:
            F_loss = F_loss * self.class_weights.to(inputs.device)  # Apply weights if provided
        
        if self.reduction == 'mean':
            return torch.mean(F_loss)
        else:
            return F_loss

def train_model(model, train_loader, val_loader, techniques, num_epochs=6, patience=2):
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)  # Increased weight_decay
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, verbose=True)
    criterion = FocalLoss(gamma=3, class_weights=class_weights)  # Increased gamma for harder examples
    best_f1 = 0.0
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_preds = []
        train_labels = []
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]"):
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
            preds = (torch.sigmoid(outputs) > 0.5).float().cpu().numpy()
            train_preds.extend(preds)
            train_labels.extend(labels.cpu().numpy())
        
        # Validation phase with threshold tuning capability
        val_metrics = evaluate_model(model, val_loader, techniques)
        
        print(f"\nEpoch {epoch+1}:")
        print(f"  Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"  Val Loss: {val_metrics['loss']:.4f}, Macro F1: {val_metrics['macro_f1']:.4f}")
        print(f"\nClassification Report for Validation Set:")
        print(val_metrics['report'])
        
        scheduler.step(val_metrics['macro_f1'])
        
        if val_metrics['macro_f1'] > best_f1:
            best_f1 = val_metrics['macro_f1']
            torch.save(model.state_dict(), "best_model.pt")
            print(f"  New best model saved with Macro F1: {best_f1:.4f}")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered after {patience} epochs without improvement")
                break
    
    # Load best model
    model.load_state_dict(torch.load("best_model.pt", weights_only=True))
    
    # Threshold tuning on validation set
    val_metrics = evaluate_model(model, val_loader, techniques)
    val_probs = val_metrics['probs']
    val_labels = val_metrics['labels']
    
    thresholds = np.arange(0.1, 0.9, 0.1)
    best_thresholds = []
    for i in range(num_classes):
        class_f1 = []
        for t in thresholds:
            preds = (val_probs[:, i] > t).astype(int)
            f1 = f1_score(val_labels[:, i], preds, zero_division=0)
            class_f1.append(f1)
        best_t = thresholds[np.argmax(class_f1)]
        best_thresholds.append(best_t)
        print(f"Best threshold for {techniques[i]}: {best_t}")
    
    return model, best_thresholds

# Updated evaluation function to return probabilities for threshold tuning
def evaluate_model(model, dataloader, techniques, threshold=0.5):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    total_loss = 0.0
    criterion = FocalLoss(gamma=3, class_weights=class_weights)
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            probs = torch.sigmoid(outputs).cpu().numpy()
            preds = (probs > threshold).astype(float)
            all_probs.extend(probs)
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    report = classification_report(all_labels, all_preds, target_names=techniques, zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    return {'loss': total_loss / len(dataloader), 'macro_f1': macro_f1, 'report': report, 'probs': all_probs, 'labels': all_labels}

# Run training and get best thresholds
model, best_thresholds = train_model(model, train_loader, val_loader, techniques, num_epochs=6, patience=2)

# Verify setup
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")


Evaluating: 100%|██████████| 96/96 [00:19<00:00,  4.86it/s]



Epoch 1:
  Train Loss: 0.0008
  Val Loss: 0.0004, Macro F1: 0.1314

Classification Report for Validation Set:
                            precision    recall  f1-score   support

           loaded_language       0.68      0.67      0.67       392
   glittering_generalities       0.74      0.36      0.49        97
                  euphoria       0.54      0.09      0.16        77
            appeal_to_fear       0.00      0.00      0.00        58
                       fud       0.00      0.00      0.00        75
                 bandwagon       0.00      0.00      0.00        35
thought_terminating_cliche       0.00      0.00      0.00         0
              whataboutism       0.00      0.00      0.00        34
            cherry_picking       0.00      0.00      0.00        97
                 straw_man       0.00      0.00      0.00        25

                 micro avg       0.68      0.34      0.45       890
                 macro avg       0.20      0.11      0.13       890
   

Evaluating: 100%|██████████| 96/96 [00:19<00:00,  4.87it/s]



Epoch 2:
  Train Loss: 0.0005
  Val Loss: 0.0004, Macro F1: 0.1508

Classification Report for Validation Set:
                            precision    recall  f1-score   support

           loaded_language       0.59      0.94      0.72       392
   glittering_generalities       0.95      0.21      0.34        97
                  euphoria       0.44      0.25      0.32        77
            appeal_to_fear       0.00      0.00      0.00        58
                       fud       0.00      0.00      0.00        75
                 bandwagon       0.15      0.11      0.13        35
thought_terminating_cliche       0.00      0.00      0.00         0
              whataboutism       0.00      0.00      0.00        34
            cherry_picking       0.00      0.00      0.00        97
                 straw_man       0.00      0.00      0.00        25

                 micro avg       0.57      0.46      0.51       890
                 macro avg       0.21      0.15      0.15       890
   

Evaluating: 100%|██████████| 96/96 [00:19<00:00,  4.86it/s]


...


Epoch 5:
  Train Loss: 0.0002
  Val Loss: 0.0007, Macro F1: 0.3285

Classification Report for Validation Set:
                            precision    recall  f1-score   support

           loaded_language       0.70      0.79      0.74       392
   glittering_generalities       0.57      0.58      0.57        97
                  euphoria       0.38      0.53      0.44        77
            appeal_to_fear       0.21      0.47      0.29        58
                       fud       0.34      0.75      0.46        75
                 bandwagon       0.13      0.20      0.16        35
thought_terminating_cliche       0.00      0.00      0.00         0
              whataboutism       0.06      0.06      0.06        34
            cherry_picking       0.36      0.41      0.38        97
                 straw_man       0.20      0.16      0.18        25

                 micro avg       0.46      0.61      0.53       890
                 macro avg       0.29      0.39      0.33       890
   

Evaluating: 100%|██████████| 96/96 [00:19<00:00,  4.87it/s]



Epoch 6:
  Train Loss: 0.0002
  Val Loss: 0.0007, Macro F1: 0.3078

Classification Report for Validation Set:
                            precision    recall  f1-score   support

           loaded_language       0.71      0.74      0.73       392
   glittering_generalities       0.54      0.65      0.59        97
                  euphoria       0.48      0.31      0.38        77
            appeal_to_fear       0.21      0.22      0.22        58
                       fud       0.46      0.43      0.44        75
                 bandwagon       0.08      0.06      0.07        35
thought_terminating_cliche       0.00      0.00      0.00         0
              whataboutism       0.00      0.00      0.00        34
            cherry_picking       0.38      0.39      0.39        97
                 straw_man       0.26      0.28      0.27        25

                 micro avg       0.55      0.53      0.54       890
                 macro avg       0.31      0.31      0.31       890
   

Evaluating: 100%|██████████| 96/96 [00:19<00:00,  4.89it/s]


Best threshold for loaded_language: 0.5
Best threshold for glittering_generalities: 0.5
Best threshold for euphoria: 0.5
Best threshold for appeal_to_fear: 0.5
Best threshold for fud: 0.6
Best threshold for bandwagon: 0.4
Best threshold for thought_terminating_cliche: 0.1
Best threshold for whataboutism: 0.2
Best threshold for cherry_picking: 0.4
Best threshold for straw_man: 0.30000000000000004
Training dataset size: 4960
Validation dataset size: 765


In [12]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm

# Load ground truth labels from solution.csv
ground_truth_df = pd.read_csv('/kaggle/input/sample/solution.csv')

# Define techniques as used in the model
techniques = [
    'loaded_language', 'glittering_generalities', 'euphoria', 'appeal_to_fear', 
    'fud', 'bandwagon', 'thought_terminating_cliche', 'whataboutism', 
    'cherry_picking', 'straw_man'
]

# Define the column order in the ground truth CSV (excluding 'Usage')
gt_columns = [
    'id', 'straw_man', 'appeal_to_fear', 'fud', 'bandwagon', 'whataboutism',
    'loaded_language', 'glittering_generalities', 'euphoria', 'cherry_picking', 'cliche'
]

# Verify that ground truth CSV has the expected columns
assert all(col in ground_truth_df.columns for col in gt_columns), "Ground truth CSV missing some columns"

# Map ground truth columns to techniques (rename 'cliche' to 'thought_terminating_cliche')
column_mapping = {
    'straw_man': 'straw_man',
    'appeal_to_fear': 'appeal_to_fear',
    'fud': 'fud',
    'bandwagon': 'bandwagon',
    'whataboutism': 'whataboutism',
    'loaded_language': 'loaded_language',
    'glittering_generalities': 'glittering_generalities',
    'euphoria': 'euphoria',
    'cherry_picking': 'cherry_picking',
    'cliche': 'thought_terminating_cliche'
}

# Ensure the ground truth CSV has the same order of IDs as test_df
# If 'Usage' indicates a split, we can filter for 'Public' or use all rows
# For now, use all rows to match test_df IDs
ground_truth_df = ground_truth_df.set_index('id').loc[test_df['id']].reset_index()

# Create a DataFrame with techniques in the model's order
ground_truth_labels = ground_truth_df.rename(columns=column_mapping)[techniques].values.astype(int)

# Create test dataset and dataloader
test_dataset = TextDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Evaluation function
def evaluate_model_with_labels(model, dataloader, techniques, best_thresholds, true_labels):
    model.eval()
    all_preds = []
    all_probs = []
    all_ids = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
            probs = torch.sigmoid(outputs).cpu().numpy()
            
            # Apply per-class thresholds
            preds = np.array([probs[:, i] > best_thresholds[i] for i in range(probs.shape[1])]).T.astype(int)
            
            all_probs.extend(probs)
            all_preds.extend(preds)
            
            # Collect IDs
            if "id" in batch:
                all_ids.extend(batch["id"].numpy().tolist())
            else:
                all_ids.extend(test_df["id"].iloc[len(all_preds)-len(probs):len(all_preds)].tolist())
    
    all_preds = np.array(all_preds)
    all_probs = np.array(all_probs)
    true_labels = np.array(true_labels)
    
    # Calculate metrics
    report = classification_report(true_labels, all_preds, target_names=techniques, zero_division=0)
    macro_f1 = f1_score(true_labels, all_preds, average='macro', zero_division=0)
    
    return {
        'preds': all_preds,
        'probs': all_probs,
        'ids': all_ids,
        'macro_f1': macro_f1,
        'report': report,
        'labels': true_labels
    }

# Run evaluation
eval_result = evaluate_model_with_labels(model, test_loader, techniques, best_thresholds, ground_truth_labels)

# Extract predictions and IDs
test_preds = eval_result['preds']
test_ids = eval_result['ids']

# Verify prediction count
if len(test_preds) != 5735:
    raise ValueError(f"Expected 5735 predictions, but got {len(test_preds)}")

# Print evaluation metrics
print("\nTest Set Evaluation Metrics:")
print(f"Macro F1: {eval_result['macro_f1']:.4f}")
print("\nClassification Report:\n", eval_result['report'])

# Print predicted label distribution
pred_counts = {tech: int(sum(test_preds[:, i])) for i, tech in enumerate(techniques)}
print("\nPredicted Label Counts for Test Set:")
print(pred_counts)

# Create submission DataFrame
submission_df = pd.DataFrame(test_preds, columns=techniques)
submission_df.insert(0, "id", test_ids)

# Reorder columns and rename for submission
submission_columns = [
    "id", "straw_man", "appeal_to_fear", "fud", "bandwagon", "whataboutism",
    "loaded_language", "glittering_generalities", "euphoria", "cherry_picking", "cliche"
]
submission_df = submission_df.rename(columns={'thought_terminating_cliche': 'cliche'})
submission_df = submission_df[submission_columns]

# Verify submission
print("\nSubmission Shape:", submission_df.shape)
if submission_df.shape[0] != 5735:
    raise ValueError(f"Submission should have 5735 rows, but found {submission_df.shape[0]}")

print("\nSubmission Head:\n", submission_df.head())

# Save submission
submission_df.to_csv("submission_with_labels.csv", index=False)
print("\nFinal submission saved as 'submission_with_labels.csv'")

Predicting: 100%|██████████| 717/717 [02:28<00:00,  4.82it/s]


Test Set Evaluation Metrics:
Macro F1: 0.3490

Classification Report:
                             precision    recall  f1-score   support

           loaded_language       0.70      0.80      0.74      2959
   glittering_generalities       0.58      0.56      0.57       723
                  euphoria       0.46      0.51      0.48       695
            appeal_to_fear       0.27      0.52      0.35       449
                       fud       0.35      0.53      0.43       576
                 bandwagon       0.12      0.28      0.17       236
thought_terminating_cliche       0.00      0.00      0.00       695
              whataboutism       0.10      0.49      0.17       235
            cherry_picking       0.28      0.66      0.40       768
                 straw_man       0.13      0.29      0.18       207

                 micro avg       0.42      0.58      0.49      7543
                 macro avg       0.30      0.46      0.35      7543
              weighted avg       0.45     